# STATISTICS Applied to data science

## Exercises PART 1: Descriptive statistics

Employing descriptive statistics is one of the main steps of the POC stage (proof of concept) and extremely helpful during model evaluation. In this notebook you'll find some common routines for descriptive statistics in Python, and exercises about data transformation and scaling. 

![Image](data_1.jpg)

### Libraries and configs

In [ ]:
import numpy as np
from numpy import random
import pandas as pd
from scipy import stats
from statsmodels.stats import power
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 20)
%matplotlib inline

from statsmodels.compat import lzip
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

# jupyter lab configs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Exercise 1 - Your own summary statistics and descriptors

Implement code for the functions below. In each function, make sure you call the function written before. E.g., in `rmse()` use the values returned by `mse()`. The aim of this exercise is just to understand how these diferent metrics are related, and which aspect of the data they are representing.   

**You can use the map below to see the relationships between metrics and then plan how to structure your functions** 

![Image](map.png)

In [ ]:
def my_mean(x):
    if len(x)>0:
        return sum(x)/len(x)

def my_sum_squares():
    pass

def my_mse():
    # mean squared error
    pass

def my_rmse():
    # rooted mean squared error
    pass

def my_variance():
    pass

def my_std_dev():
    pass

def my_std_error():
    pass

def my_confidence_95():
    pass
    
def my_covariance():
    pass

def my_coeficient_variation():
    pass

### Make sure it works!! In Python use `assert`

In [ ]:
x = random.randint(500, size=(32))
assert my_mean(x) == np.mean(x)

# Observe the properties of a Normal Distribution using cumulative probability plots

In [ ]:
stats.norm.cdf(-1)

In [ ]:
stats.norm.cdf(0)

In [ ]:
stats.norm.cdf(1) 

In [ ]:
stats.norm.cdf(2) 

In [ ]:
stats.norm.cdf(3)

What is the percentage of the curve lower than the mean minus 3 deviations?

In [ ]:
stats.norm.cdf(-3)

What is the percentage of the curve lower than the mean plus 2 sd?

In [ ]:
stats.norm.cdf(-2)

What is the percentage of the curve ABOVE the mean plus 2 sd?

In [ ]:
1 - stats.norm.cdf(2)

How many samples should we expec

The function below plots the cumulative distribution given a certain mean and standard deviation. Then change the mean and std and see what happens:

In [ ]:
def norm_cdf(mean=0.0, std=1.0):
    # 50 numbers between -3σ and 3σ
    x = np.linspace(-3*std, 3*std, 50)
    # CDF at these values
    y = stats.norm.cdf(x, loc=mean, scale=std)
    plt.ylabel("Cumulative Probability")
    plt.title("Cum. Dist. for Gaussian of mean = {0} | std. dev. = {1}".format(mean, std))
    plt.plot(x,y, color="black")
    plt.xlabel("Variate")
    plt.draw()

In [ ]:
norm_cdf()

In [ ]:
norm_cdf(mean=2, std=2.0)

In [ ]:
norm_cdf(mean=3, std=1.0)

# Data transformations

## The most common procedures are *feature scaling* and *linearization*:

1. `Scaling` means you transform the data so all quantitative features are, let's say, *speaking the same language*. We will apply `scaling` to the quantitative features. Min-max, z-score, mean normalization are very used. Particularly, I always use z-score, and this transformation is also the most common method employed in *unsupervised learning* suchs as PCA, clustering, etc.

2. `Linearization` will be usually needed to transform the `target`, or `dependent` variable, i. e., what you are trying to model

# Feature scaling (a.k.a. standardization, normalization

## Z-score transformation 

![Image](zscore.gif)
You can use `scipy.stats.zscore()` or write your own function, which is way more fun:

In [ ]:
def my_z_score(x):
    pass

# Dealing with non-gaussian data 

There's usually three ways of carrying on the analysis if you are working with regression problems and quantitative **target** variables that are not normally-distributed.
1. Look for models that don't need linear relationships in the data (E. g. random forests, boosted trees)
2. Look for models that can handle different distributions, like Poisson or Binomial (a.k.a. Generalized Linear Models)
3. If you are using a hypothesis test, use bootstrapping to generate to generate the null model 
4. Apply transformations (log, sqrt, box-cox)

**Warning**  

Log-transformation is a common tool in statistics. However, there is a pitfall in using log transformation of your data. Especially if you have a wide range of numbers, keep in mind that log will "compress" the data significantly more, and this can prevent the identification of interesting patterns

In [ ]:
# difference betwee the log and sqrt transformation of a "big" value
np.sqrt(34565)
np.log(34565)

# difference betwee the log and sqrt transformation of a "small" value
np.sqrt(107)
np.log(107)

The function below plots the diagnostic plots **QQ Plots** for two sets of variables, like raw (unstransformed) and transformed data, for comparison. 

In [ ]:
def plot_compare_transformations(raw_data, transformed_data, transformation_used):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    prob = stats.probplot(raw_data, dist=stats.norm, plot=ax1)
    ax1.set_xlabel('')
    ax1.set_title('Probplot against the normal distribution (line) ')
    ax2 = fig.add_subplot(212)
    prob = stats.probplot(transformed_data, dist=stats.norm, plot=ax2)
    ax2.set_title('Probplot after ' + transformation_used + ' transformation')
    plt.show()

### Example: 
Try **box-cox** (available in **scipy**)

In [ ]:
# generate some data with noise
raw_data = stats.loggamma.rvs(5, size=100) + 20
weird_distr_data = [1,1,1,1,1,1,1,7]
# apply box-cox
transformed_data, _ = stats.boxcox(raw_data)
# plot and compare 
plot_compare_transformations(raw_data, transformed_data, 'box-cox')

Now let's see the same effect in numbers:

Example with shapiro's test:

In [ ]:
print('Test of normal distribution with Shapiros Test')
print('stat:', stats.shapiro(raw_data)[0],'p-value:', stats.shapiro(raw_data)[1])

In [ ]:
print('Test of normal distribution with Shapiros Test')
print('stat:', stats.shapiro(transformed_data)[0],'p-value:', stats.shapiro(transformed_data)[1])

In [ ]:
name = ['Jarque-Bera', 'Chi^2 two-tail prob.', 'Skew', 'Kurtosis']
test = sms.jarque_bera(raw_data)
lzip(name, test)
test = sms.jarque_bera(transformed_data)
lzip(name, test)

Try to generate more data and apply other transformations, always inspecting the effect with QQ-plots and/or tests  

`x = np.random.normal(0, 1, 500)`

----

<a href='https://www.freepik.com/vectors/data'>Data vector created by stories - www.freepik.com</a>